In [1]:
import numpy as np
import pandas as pd
import io

In [2]:
def preprocess_lines(file_path, expected_number_of_columns):
    with open(file_path, 'r') as f:
        cleaned_lines = [line.strip() for line in f.readlines() if len(line.split(';')) == 8]
    return '\n'.join(cleaned_lines)

file_path = 'BX-Books.csv'
expected_number_of_columns = 8  # Replace with the actual expected number of columns
cleaned_data = preprocess_lines(file_path, expected_number_of_columns)

# Now use the cleaned data with read_csv
df = pd.read_csv(io.StringIO(cleaned_data),sep=';')



C:\Users\HP\AppData\Local\Temp\ipykernel_5044\633553797.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(cleaned_data),sep=';')


In [3]:
df.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [4]:
df.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [5]:
df=df[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher']]

In [6]:
df.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher'},inplace=True)

In [7]:
df.head(2)

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


In [8]:
def preprocess_lines(file_path, expected_number_of_columns):
    with open(file_path, 'r') as f:
        cleaned_lines = [line.strip() for line in f.readlines() if len(line.split(';')) == 3]
    return '\n'.join(cleaned_lines)

file_path = 'BX-Users.csv'
expected_number_of_columns = 3  # Replace with the actual expected number of columns
cleaned_data = preprocess_lines(file_path, expected_number_of_columns)

# Now use the cleaned data with read_csv
fd = pd.read_csv(io.StringIO(cleaned_data),sep=';')

In [9]:
fd.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'},inplace=True)
fd.head(2)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [10]:
def preprocess_lines(file_path, expected_number_of_columns):
    with open(file_path, 'r') as f:
        cleaned_lines = [line.strip() for line in f.readlines() if len(line.split(';')) == 3]
    return '\n'.join(cleaned_lines)

file_path = 'BX-Book-Ratings.csv'
expected_number_of_columns = 3  # Replace with the actual expected number of columns
cleaned_data = preprocess_lines(file_path, expected_number_of_columns)

# Now use the cleaned data with read_csv
rate = pd.read_csv(io.StringIO(cleaned_data),sep=';')

In [11]:
rate.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [12]:
rate.rename(columns={'User-ID':'user_id','Book-Rating':'rating'},inplace=True)

In [13]:
rate.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [14]:
#we need to reduce users as we need to make genuine real Book readers as we need to create a creteria that we need more than 200 ratings by people and more than ratings on book
#we will take books those has more than 50 ratings
flit=rate['user_id'].value_counts()>200
y=flit[flit].index

In [15]:
rate=rate[rate['user_id'].isin(y)]

In [16]:
rate.shape
rate.head(2)

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0


In [17]:
rate_book=rate.merge(df,on='ISBN')

In [18]:
rate_book.head(2)

,user_id,ISBN,rating,title,author,year,publisher
0,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning
1,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books


In [19]:
n_rate=rate_book.groupby('title')['rating'].count().reset_index()

In [20]:
n_rate.rename(columns={'rating':'number of ratings'},inplace=True)

In [21]:
n_rate.head(5)

,title,number of ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [22]:
f_rate=rate_book.merge(n_rate,on='title')

In [23]:
f_rate.shape

(463384, 8)

In [24]:
f_rate=f_rate[f_rate['number of ratings']>=50]

In [25]:
f_rate.shape

(59880, 8)

In [26]:
f_rate.drop_duplicates(['user_id','title'],inplace=True)

In [27]:
f_rate.shape

(57975, 8)

In [28]:
book_pivot=f_rate.pivot_table(columns='user_id',index='title',values='rating')

In [29]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [30]:
book_pivot.fillna(0,inplace=True)

In [31]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
#convert into sparse matrix
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(book_pivot)

In [33]:
from sklearn.neighbors import NearestNeighbors
clusters=NearestNeighbors(algorithm='brute')#To fin distance from all
clusters.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [45]:
distance,suggestions=clusters.kneighbors(book_pivot.loc['Harry Potter and the Chamber of Secrets (Book 2)',:].values.reshape(1,-1))

In [46]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333]])

In [47]:
suggestions

array([[231, 234, 232, 235, 178]], dtype=int64)

In [48]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive'],
      dtype='object', name='title')
